In [2]:

from __future__ import annotations
import argparse
import json
import logging
import threading
import time
from dataclasses import dataclass, field
from typing import List, Optional, Generator, Tuple, Callable
import sys
import os
import unittest

#confuring and logging the files for UI and access of it..
LOG_FILE = "ttt_game.log"
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)


# Exceptions if any error occurs
class InvalidMoveError(Exception):
    pass

# setting up the helpers functions for more liabilty and stability of the board and instances
def timeit(fn: Callable) -> Callable:

    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = fn(*args, **kwargs)
        end = time.perf_counter()
        logger.debug(f"{fn.__name__} ran in {(end - start):.6f}s")
        return result

    return wrapper

# setting up the data classes and modesl for designing of board
@dataclass
class Board:
    cells: List[str] = field(default_factory=lambda: [" "] * 9)

    def __post_init__(self):
        if len(self.cells) != 9:
            raise ValueError("Board must have 9 cells")

    def display(self) -> None:
        print()
        for r in range(3):
            row = " | ".join(self.cells[r * 3:(r + 1) * 3])
            print(f" {row} ")
            if r < 2:
                print("---+---+---")
        print()

    def make_move(self, index: int, symbol: str) -> None:
        if not (0 <= index < 9):
            raise InvalidMoveError("Move index must be between 0 and 8.")
        if self.cells[index] != " ":
            raise InvalidMoveError("Cell already occupied.")
        self.cells[index] = symbol

    def available_moves(self) -> Generator[int, None, None]:
        for i, v in enumerate(self.cells):
            if v == " ":
                yield i

    def is_full(self) -> bool:
        return all(cell != " " for cell in self.cells)

    def winner(self) -> Optional[str]:
        lines = [
            (0, 1, 2),
            (3, 4, 5),
            (6, 7, 8),
            (0, 3, 6),
            (1, 4, 7),
            (2, 5, 8),
            (0, 4, 8),
            (2, 4, 6),
        ]
        for a, b, c in lines:
            if self.cells[a] == self.cells[b] == self.cells[c] and self.cells[a] != " ":
                return self.cells[a]
        return None

    def clone(self) -> Board:
        return Board(self.cells.copy())

# designgin the min and max alpha beta version of game with ai
@timeit
def minimax(board: Board, depth: int, maximizing: bool, player_symbol: str, opponent_symbol: str,
            alpha: float = float("-inf"), beta: float = float("inf")) -> Tuple[int, Optional[int]]:
    win = board.winner()
    if win == player_symbol:
        return 10 - depth, None
    if win == opponent_symbol:
        return depth - 10, None
    if board.is_full():
        return 0, None

    best_move: Optional[int] = None
    if maximizing:
        max_eval = float("-inf")
        for move in board.available_moves():
            nb = board.clone()
            nb.make_move(move, player_symbol)
            score, _ = minimax(nb, depth + 1, False, player_symbol, opponent_symbol, alpha, beta)
            if score > max_eval:
                max_eval = score
                best_move = move
            alpha = max(alpha, score)
            if beta <= alpha:
                break
        return int(max_eval), best_move
    else:
        min_eval = float("inf")
        for move in board.available_moves():
            nb = board.clone()
            nb.make_move(move, opponent_symbol)
            score, _ = minimax(nb, depth + 1, True, player_symbol, opponent_symbol, alpha, beta)
            if score < min_eval:
                min_eval = score
                best_move = move
            beta = min(beta, score)
            if beta <= alpha:
                break
        return int(min_eval), best_move


# setting up the threading timer using multi-threads
class MoveTimer:
    def __init__(self, seconds: int):
        self.seconds = seconds
        self._expired = False
        self._thread: Optional[threading.Thread] = None

    def _run(self):
        logger.debug("Timer started")
        time.sleep(self.seconds)
        self._expired = True
        logger.debug("Timer expired")

    def start(self):
        self._expired = False
        self._thread = threading.Thread(target=self._run, daemon=True)
        self._thread.start()

    def cancel(self):
        self._expired = False

    def expired(self) -> bool:
        return self._expired

# saving and loading the each instance of the game in smaller instances
def save_game(board: Board, current_player: str, filename: str = "saved_game.json") -> None:
    data = {"cells": board.cells, "current_player": current_player}
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f)
    logger.info(f"Game saved to {filename}")


def load_game(filename: str = "saved_game.json") -> Tuple[Board, str]:
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    board = Board(data["cells"])
    current_player = data["current_player"]
    logger.info(f"Game loaded from {filename}")
    return board, current_player


# setting up the controllers of the game
@dataclass
class Game:
    board: Board = field(default_factory=Board)
    current_player: str = "X"
    mode: str = "hvh"
    ai_symbol: str = "O"
    timer_seconds: Optional[int] = None

    def switch(self):
        self.current_player = "O" if self.current_player == "X" else "X"

    def human_move(self, idx: int) -> None:
        self.board.make_move(idx, self.current_player)

    def ai_move(self) -> int:
        opponent_symbol = "X" if self.ai_symbol == "O" else "O"
        score, move = minimax(self.board, 0, True, self.ai_symbol, opponent_symbol)
        if move is None:
            raise RuntimeError("AI couldn't find move")
        self.board.make_move(move, self.ai_symbol)
        logger.info(f"AI placed {self.ai_symbol} at {move}")
        return move

    def check_end(self) -> Optional[str]:
        return self.board.winner()

    def play_turn(self, human_input_fn: Callable[[str], str]) -> None:
        """Play one turn depending on mode. human_input_fn is callable that asks user for input."""
        timer = MoveTimer(self.timer_seconds) if self.timer_seconds else None

        if self.mode == "hvc" and self.current_player == self.ai_symbol:
            print("AI is thinking...")
            self.ai_move()
            return
        if timer:
            timer.start()

        while True:
            if timer and timer.expired():
                print(f"Time expired! Player {self.current_player} forfeits the move.")
                logger.info(f"Player {self.current_player} timed out.")
                return
            try:
                raw = human_input_fn(f"Player {self.current_player} enter move (0-8), 's' to save, or 'q' to quit: ")
                if raw.lower() == "s":
                    save_game(self.board, self.current_player)
                    print("Game saved.")
                    continue
                if raw.lower() == "q":
                    raise KeyboardInterrupt()
                idx = int(raw.strip())
                self.human_move(idx)
                logger.info(f"Player {self.current_player} moved to {idx}")
                break
            except InvalidMoveError as e:
                print(f"Invalid move: {e}")
            except ValueError:
                print("Please enter a valid number 0-8, 's', or 'q'.")

        if timer:
            timer.cancel()

    def run(self, human_input_fn: Callable[[str], str]) -> None:
        """Main game loop (console)."""
        logger.info("New game started")
        while True:
            self.board.display()
            winner = self.check_end()
            if winner or self.board.is_full():
                self.board.display()
                if winner:
                    print(f"Winner: {winner}!")
                    logger.info(f"Winner: {winner}")
                else:
                    print("It's a tie!")
                    logger.info("Game ended in a tie")
                break

            print(f"Current player: {self.current_player}")
            self.play_turn(human_input_fn)
            if self.mode == "hvc" and self.current_player == self.ai_symbol:
                pass
            else:
                self.switch()

# setting up the UI with CLI helpers functions
def input_wrapper(prompt: str) -> str:
    return input(prompt)


def menu() -> None:
    print("Advanced Tic-Tac-Toe")
    print("====================")
    print("1. Human vs Human")
    print("2. Human vs AI")
    print("3. Load saved game")
    print("4. Quit")


def interactive_main(args) -> None:
    while True:
        menu()
        choice = input("Choose option: ").strip()
        if choice == "1":
            game = Game(mode="hvh", current_player="X", timer_seconds=(args.timer or None))
            game.run(input_wrapper)
        elif choice == "2":
            first = input("Do you want to be X (go first)? (y/n): ").strip().lower()
            if first == "y":
                human_symbol = "X"
                ai_symbol = "O"
                current = "X"
            else:
                human_symbol = "O"
                ai_symbol = "X"
                current = "X"
            game = Game(mode="hvc", current_player=current, ai_symbol=ai_symbol, timer_seconds=(args.timer or None))
            def human_input(prompt: str) -> str:
                if game.current_player == game.ai_symbol:
                    return "0"
                return input(prompt)
            try:
                game.run(human_input)
            except KeyboardInterrupt:
                print("Exiting to main menu.")
        elif choice == "3":
            fname = input("Filename (default saved_game.json): ").strip() or "saved_game.json"
            if not os.path.exists(fname):
                print(f"No such file: {fname}")
                continue
            b, current = load_game(fname)
            mode = input("Mode for this session? (hvh/hvc) [hvh]: ").strip() or "hvh"
            ai_symbol = "O"
            timer = args.timer or None
            g = Game(board=b, current_player=current, mode=mode, ai_symbol=ai_symbol, timer_seconds=timer)
            try:
                g.run(input_wrapper)
            except KeyboardInterrupt:
                print("Returning to menu.")
        elif choice == "4":
            print("Goodbye.")
            break
        else:
            print("Invalid option.")

# building unit tests for small instance of game
class TestBoard(unittest.TestCase):
    def test_winner_rows(self):
        b = Board(["X", "X", "X", " ", " ", " ", " ", " ", " "])
        self.assertEqual(b.winner(), "X")

    def test_available_moves(self):
        b = Board(["X", " ", "O", " ", " ", " ", " ", " ", " "])
        moves = list(b.available_moves())
        self.assertIn(1, moves)
        self.assertIn(3, moves)
        self.assertNotIn(0, moves)

    def test_make_move_invalid(self):
        b = Board()
        b.make_move(0, "X")
        with self.assertRaises(InvalidMoveError):
            b.make_move(0, "O")

    def test_minimax_block(self):
        b = Board(["X", "X", " ", " ", "O", " ", " ", " ", " "])
        score, move = minimax(b, 0, True, "O", "X")
        self.assertEqual(move, 2)


def run_tests():
    print("Running unit tests...")
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(TestBoard)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    if not result.wasSuccessful():
        sys.exit(1)


# making sure entry point and argparse is in correct
def build_parser():
    p = argparse.ArgumentParser(description="Advanced Tic-Tac-Toe single-file project")
    p.add_argument("--test", action="store_true", help="Run unit tests and exit")
    p.add_argument("--timer", type=int, help="Set per-move timer in seconds")
    return p


def main():
    parser = build_parser()
    args , unknown = parser.parse_known_args()
    if args.test:
        run_tests()
        return
    try:
        interactive_main(args)
    except KeyboardInterrupt:
        print("\nExiting. Bye!")
    except Exception as e:
        logger.exception("Unhandled exception")
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()


Advanced Tic-Tac-Toe
1. Human vs Human
2. Human vs AI
3. Load saved game
4. Quit
Choose option: 2
Do you want to be X (go first)? (y/n): y

   |   |   
---+---+---
   |   |   
---+---+---
   |   |   

Current player: X
Player X enter move (0-8), 's' to save, or 'q' to quit: 5

   |   |   
---+---+---
   |   | X 
---+---+---
   |   |   

Current player: O
AI is thinking...

   |   | O 
---+---+---
   |   | X 
---+---+---
   |   |   

Current player: O
AI is thinking...

 O |   | O 
---+---+---
   |   | X 
---+---+---
   |   |   

Current player: O
AI is thinking...

 O | O | O 
---+---+---
   |   | X 
---+---+---
   |   |   


 O | O | O 
---+---+---
   |   | X 
---+---+---
   |   |   

Winner: O!
Advanced Tic-Tac-Toe
1. Human vs Human
2. Human vs AI
3. Load saved game
4. Quit
Choose option: 1

   |   |   
---+---+---
   |   |   
---+---+---
   |   |   

Current player: X
Player X enter move (0-8), 's' to save, or 'q' to quit: 4

   |   |   
---+---+---
   | X |   
---+---+---
   |   | 